### Project 1

TimeZone Class

In [11]:
import numbers
from datetime import timedelta

In [14]:
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0: 
            raise ValueError('TimeZone name cannot be empty')

        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minutes offset must be an integer")

        if abs(offset_minutes) > 59 :
            raise ValueError("Minutes offset must be [-59, 59].")

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError("Offset must be between -12.00 and 14.00")

        self._offest_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return(isinstance(other, TimeZone) and
               self.name == other.name and
               self._offest_hours == other._offset_hours and
               self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return(f"TimeZone(name='{self.name}', "
               f"offest_hours={self._offset_hours}, "
               f"offset_minutes={self._offset_minutes})")


In [16]:
tz1 = TimeZone("ABC", -2, -15)
tz1.name

'ABC'

In [18]:
from datetime import datetime

dt = datetime.utcnow()
print(dt)

2023-05-08 11:17:17.804017


In [19]:
print(dt + tz1.offset)

2023-05-08 09:02:17.804017


In [21]:
try:
    tz = TimeZone("", 1, 10)
except ValueError as ex:
    print(ex)

TimeZone name cannot be empty
